# EchoWare

Retarget an expert pose to an agent pose

The expert and agent poses consist of a configuration space, set of links and a set of joint transforms.
$
    \{\mathbb{C}_e,\mathbb{L}_e,\mathbb{J}_e\} \rightarrow \{\mathbb{C}_a,\mathbb{L}_a,\mathbb{J}_a\}
$

Where

$ \mathbb{C} $: Configuration, DOF x 1 vector 

$ \mathbb{L} $: Link set, {(Polygon3D, &Joint, ...), ...} 

$ \mathbb{J} $: Joint set, {(Translation3D, state, Axis3D, effect, &Link), ...}


> **Configuration** of any robot can and should be implemented as a list of actuator states. This vector is the input to forward kinematics and the output of inverse kinematics.  

> **Links** provide a geometric definition for the parts of a robot. A link consists of a polygon (convex hull of 3D points) and a possibly empty set of joint references.

> **Joints** describe a state, axis and motion of the joint on the effected link. Most joints will be continuous revolute about their Z axis (standard way to define motor reference frames). Each joint refers to a single link, creating chains: link -> joint -> link -> joint -> link. The axis of the joint is a normal vector oriented along the axis of rotation (revolute) or linear motion (linear).

For this project we will assume a set of rules the expert and agent must follow:
- A Link can have any number of Joints
- No set of links/joints forms a cycle
- All chains end in a Link/Every joint has a base link and an effected link
- A proper mapping should be independant of configuration


## Example Definition
![sample robot arm](two_link_basic.png)
> 2 DOF, 3 link robot arm (joints at $P_1$ and $P_2$, ee at $P_3$)

The above robot has the configuration space $$ \begin{bmatrix}\theta_1 \\ \theta_2 \end{bmatrix} $$

The linkages can be defined as line segments connecting the two joints and end effector: $$ \{ Link_0: ((P_0, P_1), Joint_1), Link_1: ((P_1, P_2), Joint_2),  Link_2: ((P_2, P_3), NULL)\} $$
*the points in this definition are defined in the link's reference frame. This means the first point in the segment is always at the origin and the second point is the length of the link along the X axis. The true orientation and position of the links is found using kinematics and requires knowledge of the joint states*

The Joints are offset from the links by the line segment that defines the links. This allows a very simple Transform that translates by the vector difference of the end points (or the length of the link along the X axis). $$ \{Joint_1: (P_1-P_0, \theta_1, [0,0,1], revolute, Link_1), Joint_2: (P_2-P_1, \theta_2, [0,0,1], revolute, Link_2)\} $$

### Forward Kinematics
In order to solve the forward kinematics for the end effector, start with a vector from $Link_n$ to the end effector ($P_3-P_2$ in this case). Apply the effect of $Joint_n$ to this vector (rotation about Z) and add the transform to $Link_{n-1}$ ($P_2-P_1$), this is the end effector defined relative to $Link_{n-1}$. Repeat this process untill the end effector is defined about $Link_0$.

## Retargeting Goals
To retarget this pose we should consider what the goal of retargeting is. Ideally the agent can reproduce results of an expert's actions. Due to the complexity of considering the environment, this project simplifies the goal to imitating an expert's pose. Work done by students at CMU showed that imitating an experts pose well enough can lead to an agent accomplishing tasks. Future work should require an agent also diverge from the expert enough to be in a feasible pose. This addition would look like physical constraints for the retargeting (no self collisions, match COM/momentum/forces).

## Retargeting Delivarables
An efficient and easy to implement mapping that can represent the result of this algorithm is the matrix $ \mathbb{M}_e^a $. Which is implemented as $ \mathbb{C}_a = \mathbb{M}_e^a * \mathbb{C}_e $. This claims any joint on the agent is a linear combiniation of joints on the expert. Also note $ \mathbb{M}_e^a \in \mathbb{R}^{n,m}$, where n is the agent DOF and m is the expert DOF (ie a cell relating each expert and agent joint). The algorithm below will descibe how to asssign weights to the cells of $ \mathbb{M}_e^a $.


## Joint/Link Matching
*Joints with the same depth/transform will match* 

    // does not require kinematics or optimization
    // uses thresholds and distances for similarities
    // subject to biasing
    match_joints(link_e, link_a, M):
        
        for joint_e in link_e:
        
            for joint_a in link_a:

                // joint is defined with a similar orientation and translation
                // always between 0 and 1, but thrown out if < joint threshold
                let weight = joint_e ~= joint_a
                
                if weight > joint_threshold && link_e ~= link_a > link_threshold:

                    M(joint_e, joint_a) = weight

                    // match_joints(link_e, joint_a.link, M) // try adjacent links
                    
                    match_joints(joint_e.link, joint_a.link, M) // same depth links
                    
                    // match_joints(joint_e.link, link_a, M) // try adjacent links
                    
    M = zeros(n,m) // default weights of zero for all joint relationships
    match_joints(expert_base, agent_base, M)
                

    
Beginning from the expert's base link examine the 0 depth joints (connected to the base). Matching Joints are: the same depth, close in proximity (euclidean distance, orientation threashold, effect) relative to their parent link and have a similar effected link (inertia/vertices/faces?). An agent's joint can match with multiple joints on an expert (but not vise versa) resulting in a weighted average (this could get messy...). Continue matching joints at each depth (joints only match if their parents match).

> if the expert chain has a different number of links, the agent will partially mimic that chain (maybe allow joints to relate to joints that are proportionally deep in the chain, underactuated maps => average of many, overactuated maps => one to many)

### End Effector/Chain matching
*Chains will match end effector positions* \
Begin with the expert's base link and match any joints (ignoring links). Then select from the following for each joint:
1. Use IK of agent to create similar ee position (likely doesn't use the same shape)
2. Use optimization to minimize the error in keypoints on the chain
3. Use the closing circle (sphere) algorithm (put all joint positions and desired ee on a circle and shrink the circle until they meet)

> Both require normalizing each chain definition so numerically chains have the same reach

In [1]:
# This allows file changes to be included without reloading the kernel
# Still requires reimporting the modules

%load_ext autoreload 
%autoreload 2   

In [34]:
import numpy as np
from pose import Transform3D

In [32]:
tf = Transform3D(1,0,0,1.57,-3.1415,1.57)

In [33]:
v = np.array([1 ,0 ,0])
tf.transform(v)

array([2.66453526e-15, 3.41810697e-12, 7.37825283e-08])